# COMP3009 Cw2 – Data Pre-processing



- TrainDataset_before_process.csv (400 patients)


In this notebook we:
1. Load the training dataset and inspect its dimensions and basic structure.
2. Handle missing data (values encoded as 999).
3. Separate:
   - patient ID
   - targets (PCR, RFS)
   - feature matrix
4. Identify numerical vs categorical features.
5. Build a pre-processing transformer:
   - numeric: median imputation + standardisation
   - categorical: most frequent imputation + one-hot encoding
6. Apply the pre-processing to obtain clean feature matrices ready for modelling.



In [ ]:
import pandas as pd
import os


input_file = "Dataset/TrainDataset2025-2.xls"
output_file = "Dataset/dataset.csv"

if not os.path.exists(output_file):
    print(f"Converting {input_file} → {output_file} ...")
    df_raw = pd.read_excel(input_file)
    df_raw.to_csv(output_file, index=False)
    print("Conversion complete.")
else:
    print(f"{output_file} already exists — skipping conversion.")

Dataset/dataset.csv already exists — skipping conversion.


In [1]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

## 1. Load the training dataset and inspect dimensions

We now:
- load TrainDataset_before_process.csv into a pandas DataFrame,
- check the shape(number of rows and columns),
- look at the first few rows to understand the structure.

In [3]:
train_path = "/Users/remylieberman/Desktop/code/MLcw2/Dataset/dataset.csv"  

df = pd.read_csv(train_path)

print("Training dataset shape (rows, columns):", df.shape)
df.head()

Training dataset shape (rows, columns): (400, 121)


,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.0,41.0,0,0,0,1,3,3,...,0.517172,0.375126,3.325332,0.002314,3880771.500,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.0,39.0,1,1,0,0,3,3,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.0,31.0,0,0,0,1,2,1,...,0.534549,0.534549,2.485848,0.006752,1540027.421,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.0,35.0,0,0,0,1,3,3,...,0.506185,0.506185,2.606255,0.003755,6936740.794,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.0,61.0,1,0,0,0,2,1,...,0.462282,0.462282,2.809279,0.006521,1265399.054,39.621023,0.006585,0.034148,0.001083,0.005626


We next inspect:
- the column names,
- the data types,

to get an idea of which columns are clinical, which are MRI features,
and which might be targets (PCR, RFS) or IDs.

In [4]:
print("Column names:")
print(df.columns.tolist())

print("\nData types:")
print(df.dtypes)

Column names:
['ID', 'pCR (outcome)', 'RelapseFreeSurvival (outcome)', 'Age', 'ER', 'PgR', 'HER2', 'TrippleNegative', 'ChemoGrade', 'Proliferation', 'HistologyType', 'LNStatus', 'TumourStage', 'Gene', 'original_shape_Elongation', 'original_shape_Flatness', 'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength', 'original_shape_Maximum2DDiameterColumn', 'original_shape_Maximum2DDiameterRow', 'original_shape_Maximum2DDiameterSlice', 'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume', 'original_shape_MinorAxisLength', 'original_shape_Sphericity', 'original_shape_SurfaceArea', 'original_shape_SurfaceVolumeRatio', 'original_shape_VoxelVolume', 'original_firstorder_10Percentile', 'original_firstorder_90Percentile', 'original_firstorder_Energy', 'original_firstorder_Entropy', 'original_firstorder_InterquartileRange', 'original_firstorder_Kurtosis', 'original_firstorder_Maximum', 'original_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Mean', 'original_firstor

## 2. Identify ID and target columns

From the coursework description, the dataset should contain:
- A patient ID column
- A PCR column (classification target),
- An RFS column (regression target),
- 11 clinical features,
- 107 MRI features.



In [ ]:
ID_COL = "ID"
PCR_COL = "pCR (outcome)"
RFS_COL = "RelapseFreeSurvival (outcome)"

for col in [ID_COL, PCR_COL, RFS_COL]:
    if col not in df.columns:
        print(f"WARNING: column '{col}' not found in DataFrame columns.")

## 3. Handle missing data (999 → NaN)

In [29]:
# Count how many 999 values exist before replacement
num_999_before = (df == 999).sum().sum()
print("Total '999' entries before replacement:", num_999_before)

# Replace 999 with NaN
df = df.replace(999, np.nan)

# Check how many NaNs we have now
num_nan_after = df.isna().sum().sum()
print("Total NaN entries after replacement:", num_nan_after)

# Optional: show columns with most missing values
missing_per_col = df.isna().sum().sort_values(ascending=False)
missing_per_col.head(15)

Total '999' entries before replacement: 0
Total NaN entries after replacement: 105


Gene                                                  88
pCR (outcome)                                          5
ChemoGrade                                             3
HistologyType                                          3
Proliferation                                          2
LNStatus                                               1
PgR                                                    1
HER2                                                   1
TrippleNegative                                        1
original_glrlm_GrayLevelVariance                       0
original_gldm_SmallDependenceEmphasis                  0
original_gldm_SmallDependenceHighGrayLevelEmphasis     0
original_gldm_SmallDependenceLowGrayLevelEmphasis      0
original_glrlm_GrayLevelNonUniformity                  0
original_glrlm_GrayLevelNonUniformityNormalized        0
dtype: int64

## 4. Split into ID, targets, and feature matrix

We now separate:
- patient_id: ID column (not used as a feature),
- y_pcr: PCR label (classification target),
- y_rfs: RFS value (regression target),
- X_full: all feature columns used as input to the models (clinical + MRI).

In [ ]:
patient_id = df[ID_COL]
y_pcr = df[PCR_COL]
y_rfs = df[RFS_COL]

X_full = df.drop(columns=[ID_COL, PCR_COL, RFS_COL])

print("Feature matrix shape:", X_full.shape)
print("PCR target shape:", y_pcr.shape)
print("RFS target shape:", y_rfs.shape)

Feature matrix shape: (400, 118)
PCR target shape: (400,)
RFS target shape: (400,)


## 5. Identify clinical vs MRI features

From the assignment brief, there are:
- **11 clinical features**:
  - Age, ER, PgR, HER2, TripleNegative status, Chemotherapy Grade,
    Tumour Proliferation, Histology Type, Lymph node Status,
    Tumour Stage, Gene
- **107 MRI-based features** extracted from Pyradiomics.

In [ ]:
clinical_features = [
    "Age",
    "ER",
    "PgR",                
    "HER2",
    "TrippleNegative",
    "ChemoGrade",
    "Proliferation",
    "HistologyType",
    "LNStatus",
    "TumourStage",
    "Gene",
]

missing_clinical = [c for c in clinical_features if c not in X_full.columns]
if missing_clinical:
    print("WARNING: The following clinical feature names not found in X_full:")
    print(missing_clinical)

mri_features = [c for c in X_full.columns if c not in clinical_features]

print(f"Number of clinical features (expected up to 11): {len(clinical_features) - len(missing_clinical)}")
print(f"Number of MRI features (expected around 107): {len(mri_features)}")

Number of clinical features (expected up to 11): 11
Number of MRI features (expected around 107): 107


## 6. Identify numerical and categorical input features

For pre-processing we further classify features into:
- **Numeric** (int/float): suitable for median imputation and standardisation.
- **Categorical** (e.g. strings, codes): suitable for most-frequent imputation and one-hot encoding.

In [14]:
numeric_features = X_full.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X_full.select_dtypes(exclude=["int64", "float64"]).columns.tolist()

print("Number of numeric features:", len(numeric_features))
print("Number of categorical features:", len(categorical_features))
print("\nExample numeric features:", numeric_features[:10])
print("Example categorical features:", categorical_features[:10])

Number of numeric features: 118
Number of categorical features: 0

Example numeric features: ['Age', 'ER', 'PgR', 'HER2', 'TrippleNegative', 'ChemoGrade', 'Proliferation', 'HistologyType', 'LNStatus', 'TumourStage']
Example categorical features: []


## 7. Build a pre-processing transformer

We now create a **ColumnTransformer**


In [15]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Age', 'ER', 'PgR', 'HER2', 'TrippleNegative',
                                  'ChemoGrade', 'Proliferation',
                                  'HistologyType', 'LNStatus', 'TumourStage',
                                  'Gene', 'original_shape_Elongation',
                                  'original_shape_Flatness',
                                  'original_shape_LeastAxisLength',
                                  'original_sha...
                                  'original_shape_SurfaceVolumeRatio',
                                  'original_shape_VoxelVolume',
                                  'original_firstorder_10Percentile',
                                  'original_firstorder_90Percentile',
                                  'original_firstorder_Energy',
                                  'original_firstorder_Entropy',
                                  'original_firstorder_InterquartileRange', ...]),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 [])])

## 8. Fit the pre-processor and transform the features

We now fit the pre-processing pipeline on the full training feature matrix X_full



 the pre-processing pipeline on the full training feature matrix x_full
a


In [16]:
X_processed = preprocessor.fit_transform(X_full)

print("Processed feature matrix type:", type(X_processed))
print("Processed feature matrix shape:", X_processed.shape)

Processed feature matrix type: <class 'numpy.ndarray'>
Processed feature matrix shape: (400, 118)


In [ ]:
from sklearn.impute import SimpleImputer
import pandas as pd


feature_cols = [c for c in df.columns if c not in [ID_COL, PCR_COL, RFS_COL]]

X_full = df[feature_cols].copy()


all_nan_cols = X_full.columns[X_full.isna().all()]
if len(all_nan_cols) > 0:
    print("Dropping completely empty feature columns:", list(all_nan_cols))
    X_full = X_full.drop(columns=all_nan_cols)
    feature_cols = X_full.columns.tolist()  # update feature list

numeric_features = X_full.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X_full.select_dtypes(exclude=["int64", "float64"]).columns.tolist()

print("Numeric features:", len(numeric_features))
print("Categorical features:", len(categorical_features))


df_clean = X_full.copy()

if numeric_features:
    num_imputer = SimpleImputer(strategy="median")
    df_clean[numeric_features] = num_imputer.fit_transform(df_clean[numeric_features])

if categorical_features:
    cat_imputer = SimpleImputer(strategy="most_frequent")
    df_clean[categorical_features] = cat_imputer.fit_transform(df_clean[categorical_features])

print("Remaining NaNs in features:", df_clean.isna().sum().sum())


df_clean["ID"] = patient_id.values
df_clean["pCR (outcome)"] = y_pcr.values
df_clean["RelapseFreeSurvival (outcome)"] = y_rfs.values

cols_in_order = [c for c in df.columns if c in df_clean.columns]
df_clean = df_clean[cols_in_order]


output_path = "Dataset/preprocessed_dataset.csv"
df_clean.to_csv(output_path, index=False)

print(f"Preprocessed dataset saved to:\n{output_path}")

Numeric features: 118
Categorical features: 0
Remaining NaNs in features: 0
Preprocessed dataset saved to:
Dataset/preprocessed_dataset.csv


/var/folders/2c/swvbhb6s2nv6gz94ctr_j93c0000gn/T/ipykernel_23621/656098034.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_clean["ID"] = patient_id.values
/var/folders/2c/swvbhb6s2nv6gz94ctr_j93c0000gn/T/ipykernel_23621/656098034.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_clean["pCR (outcome)"] = y_pcr.values
/var/folders/2c/swvbhb6s2nv6gz94ctr_j93c0000gn/T/ipykernel_23621/656098034.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

In [31]:
import pandas as pd

df_full = pd.read_csv("Dataset/preprocessed_dataset.csv")

protected_cols = ["ID", "pCR (outcome)", "RelapseFreeSurvival (outcome)"]

cols_with_missing = df_full.columns[df_full.isna().any()].tolist()

cols_to_drop = [col for col in cols_with_missing if col not in protected_cols]

print("Columns that will be dropped because they contain missing values:")
print(cols_to_drop)

df_cleaned = df_full.drop(columns=cols_to_drop)

print("\nNew shape after removing incomplete feature columns:", df_cleaned.shape)

output_path = "Dataset/preprocessed_dataset_cleaned.csv"
df_cleaned.to_csv(output_path, index=False)

print(f"\nCleaned dataset saved to:\n{output_path}")

Columns that will be dropped because they contain missing values:
[]

New shape after removing incomplete feature columns: (400, 121)

Cleaned dataset saved to:
Dataset/preprocessed_dataset_cleaned.csv


In [ ]:
import pandas as pd
import numpy as np

check_df = pd.read_csv("Dataset/preprocessed_dataset.csv")

print("Dataset loaded for validation.")
print("Shape:", check_df.shape)

total_nans = check_df.isna().sum().sum()
print(f"\nTotal NaN values in dataset: {total_nans}")

cols_with_nan = check_df.columns[check_df.isna().any()].tolist()
print("\nColumns with missing (NaN) values:")
print(cols_with_nan)

print("\nNaN count per column:")
print(check_df.isna().sum()[check_df.isna().sum() > 0])

blank_mask = (check_df.applymap(lambda x: isinstance(x, str) and x.strip() == ""))
total_blanks = blank_mask.sum().sum()
print(f"\nTotal blank-string ('') entries: {total_blanks}")

cols_with_blank = blank_mask.columns[blank_mask.any()].tolist()
print("\nColumns containing blank values (empty strings):")
print(cols_with_blank)

print("\nBlank count per column:")
print(blank_mask.sum()[blank_mask.sum() > 0])


empty_cols = check_df.columns[(check_df.isna().all()) | (blank_mask.all())].tolist()
print("\nColumns that are completely empty (all NaN or all blank):")
print(empty_cols)

rows_with_missing = check_df[check_df.isna().any(axis=1)]
print(f"\nNumber of rows with missing values: {rows_with_missing.shape[0]}")

print("\nValidation complete.")

Dataset loaded for validation.
Shape: (400, 121)

Total NaN values in dataset: 5

Columns with missing (NaN) values:
['pCR (outcome)']

NaN count per column:
pCR (outcome)    5
dtype: int64

Total blank-string ('') entries: 0

Columns containing blank values (empty strings):
[]

Blank count per column:
Series([], dtype: int64)

Columns that are completely empty (all NaN or all blank):
[]

Number of rows with missing values: 5

Validation complete.


### Validation Summary

The validation checks confirm that all clinical and MRI features have been fully cleaned and imputed, with no remaining NaN or blank values. No columns are completely empty. The only missing values in the dataset are 5 entries in the PCR outcome column, which indicates that these patients simply have no recorded PCR label. These rows will be removed only for the PCR classification task. All other data is complete and ready for feature selection and modelling.

we have also created a new dataset csv with the updated data, that can be used in diferent sections
all datarows with missing data that mostfrqeuent and mediun couldnt support, were removed.

